In [10]:
from scipy.io import wavfile

In [62]:
from scipy import signal

In [17]:
import numpy as np

In [31]:
import librosa

In [64]:
# General algorithm parameters
order = 12

In [65]:
# Open the WAV file used as the voice input
fs, x = wavfile.read('voice_1.wav')

In [66]:
# Voice Framing
# Create data frames of ~ 10ms to ensure local stationarity of the voice,
# this way the statistical parameters of the random signal remain the same 
# under the analysis window.
frame_time = 10e-3
frame_size = int(fs * frame_time)
x_size = len(x)
x_frames = librosa.util.frame(
    x, 
    frame_length=int(np.floor(x_size/frame_size)), 
    hop_length=frame_size
)

In [73]:
# Short-Time Autocorrelation
# Estimate the short-time autocorrelation of the frame
rxx = signal.correlate(x_frames[0,:], x_frames[0,:-order-1], method='fft') / len(x)
rxx = rxx[len(rxx) // 2 : len(rxx) // 2 + order + 1]

13